In [ ]:
# 필요한 라이브러리 임포트
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import time
import requests
import json

# 모듈 경로 추가
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # notebooks의 상위 디렉토리 = 04_건축HUB_API_matching
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # 프로젝트 루트 디렉토리

# 모듈 경로 추가
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

# 프로젝트 루트의 .env 파일 로드
env_path = os.path.join(project_root, ".env")
load_dotenv(env_path)

# api_client 모듈 임포트
try:
    from src.api.api_client import BuildingLedgerClient
    
    # 환경변수에서 API 키를 사용하여 클라이언트 생성
    client = BuildingLedgerClient()
    
    # 단일 건축물 정보 조회
    sigungu_code = "41135"  # 성남시 분당구
    bdong_code = "11000"    # 백현동
    bun = "542"
    ji = ""
    
    # 데이터 조회 시도 함수
    def try_get_data(func, *args, **kwargs):
        max_retries = 3
        retry_delay = 2
        
        for attempt in range(max_retries):
            try:
                result = func(*args, **kwargs)
                if not result.empty:
                    return result, True
                else:
                    print(f"데이터가 비어 있습니다. 재시도 중... ({attempt+1}/{max_retries})")
            except Exception as e:
                print(f"오류 발생: {e}. 재시도 중... ({attempt+1}/{max_retries})")
            
            time.sleep(retry_delay)
        
        return pd.DataFrame(), False
    
    # 표제부 데이터 조회
    print("표제부 데이터 조회 중...")
    표제부_df, 표제부_성공 = try_get_data(
        client.get_building_data,
        ledger_type="표제부",
        sigungu_code=sigungu_code,
        bdong_code=bdong_code,
        bun=bun,
        ji=ji
    )
    
    # 결과 확인
    if 표제부_성공:
        print(f"표제부 데이터 조회 성공: {len(표제부_df)}개 레코드")
        display(표제부_df.head())  # Jupyter에서 데이터프레임 표시
        
        # 컬럼 확인
        print("\n표제부 데이터 컬럼:")
        for col in 표제부_df.columns:
            print(f"- {col}")
    else:
        print("표제부 데이터 조회 실패 - API 응답 확인 필요")
        
        # API 직접 호출로 문제 확인
        print("\nAPI 직접 호출 시도...")
        api_key = os.getenv("BUILDING_HUB_API_KEY")
        if not api_key:
            print("API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")
        else:
            try:
                url = "http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo"
                params = {
                    'serviceKey': api_key,
                    'sigunguCd': sigungu_code,
                    'bjdongCd': bdong_code,
                    'bun': bun,
                    'ji': ji,
                    'numOfRows': '10',
                    'pageNo': '1',
                    'format': 'json'
                }
                response = requests.get(url, params=params)
                print(f"API 응답 상태 코드: {response.status_code}")
                print(f"API 응답 내용: {response.text[:500]}...")  # 응답 내용 일부 출력
            except Exception as e:
                print(f"API 직접 호출 중 오류: {e}")
    
    # 총괄표제부 데이터 조회
    print("\n총괄표제부 데이터 조회 중...")
    총괄표제부_df, 총괄표제부_성공 = try_get_data(
        client.get_building_data,
        ledger_type="총괄표제부",
        sigungu_code=sigungu_code,
        bdong_code=bdong_code,
        bun=bun,
        ji=ji
    )
    
    # 결과 확인
    if 총괄표제부_성공:
        print(f"총괄표제부 데이터 조회 성공: {len(총괄표제부_df)}개 레코드")
        display(총괄표제부_df.head())
    else:
        print("총괄표제부 데이터 조회 실패 - 다른 건물 ID로 시도해 보세요")
    
    # 기본개요 데이터 조회
    print("\n기본개요 데이터 조회 중...")
    기본개요_df, 기본개요_성공 = try_get_data(
        client.get_building_data,
        ledger_type="기본개요",
        sigungu_code=sigungu_code,
        bdong_code=bdong_code,
        bun=bun,
        ji=ji
    )
    
    # 결과 확인
    if 기본개요_성공:
        print(f"기본개요 데이터 조회 성공: {len(기본개요_df)}개 레코드")
        display(기본개요_df.head())
    else:
        print("기본개요 데이터 조회 실패 - 다른 건물 ID로 시도해 보세요")
    
    # 여러 건물 정보 일괄 조회 - 다양한 건물 ID 시도
    target_buildings = [
        {
            'sigungu_code': '41135',  # 성남시 분당구
            'bdong_code': '11000',    # 백현동
            'bun': '542',
            'ji': ''
        },
        {
            'sigungu_code': '11680',  # 서울시 강남구
            'bdong_code': '10300',    # 역삼동
            'bun': '100',
            'ji': '5'
        },
        {
            'sigungu_code': '11680',  # 서울시 강남구
            'bdong_code': '10300',    # 역삼동
            'bun': '737',
            'ji': '1'
        },
        {
            'sigungu_code': '11680',  # 서울시 강남구
            'bdong_code': '10300',    # 역삼동
            'bun': '825',
            'ji': '155'
        }
    ]
    
    # 표제부 정보 일괄 조회
    print("\n여러 건물 표제부 정보 일괄 조회 중...")
    try:
        batch_data = client.batch_get_building_data(
            ledger_type="표제부",
            target_buildings=target_buildings,
            delay=2  # 딜레이 증가
        )
        
        # 결과 확인
        if not batch_data.empty:
            print(f"일괄 조회 성공: {len(batch_data)}개 레코드")
            display(batch_data.head())
            
            # 결과 저장
            save_path = os.path.join(notebook_dir, "data", "batch_building_data.csv")
            batch_data.to_csv(save_path, index=False, encoding='utf-8-sig')
            print(f"\n일괄 조회 데이터 저장 완료: {save_path}")
        else:
            print("일괄 조회 실패 또는 데이터 없음 - API 호출 한도를 확인하세요")
    except Exception as e:
        print(f"일괄 조회 중 오류 발생: {e}")
    
    # 데이터 시각화 (표제부 데이터가 있는 경우)
    if 표제부_성공 and '연면적' in 표제부_df.columns:
        try:
            plt.figure(figsize=(10, 6))
            연면적_데이터 = pd.to_numeric(표제부_df['연면적'], errors='coerce')
            plt.hist(연면적_데이터.dropna(), bins=10, alpha=0.7, color='skyblue', edgecolor='black')
            plt.title('건축물 연면적 분포')
            plt.xlabel('연면적(㎡)')
            plt.ylabel('빈도')
            plt.grid(axis='y', alpha=0.75)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"시각화 중 오류 발생: {e}")

except ImportError:
    print("BuildingLedgerClient 모듈을 찾을 수 없습니다. src/api/api_client.py 파일이 존재하는지 확인하세요.")
except Exception as e:
    print(f"초기화 중 오류 발생: {e}")
    print("\n문제 해결 방법:")
    print("1. API 키가 올바르게 설정되었는지 확인하세요 (.env 파일)")
    print("2. 인터넷 연결을 확인하세요")
    print("3. API 서비스가 정상 작동 중인지 확인하세요 (공공데이터포털 공지사항 확인)")
    print("4. API 일일 호출 한도를 초과하지 않았는지 확인하세요")

표제부 데이터 조회 중...
데이터가 비어 있습니다. 재시도 중... (1/3)
데이터가 비어 있습니다. 재시도 중... (2/3)
데이터가 비어 있습니다. 재시도 중... (3/3)
표제부 데이터 조회 실패 - API 응답 확인 필요

API 직접 호출 시도...
API 응답 상태 코드: 500
API 응답 내용: <?xml version="1.0" encoding="UTF-8"?>
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
    <soapenv:Body>
        <soapenv:Fault>
            <faultcode>soapenv:Server</faultcode>
            <faultstring>Policy Falsified</faultstring>
            <faultactor>http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?serviceKey=27fTvKBXz3JSQcaB8oDKMI0NBzItDntrjY%2FZc78FBUw7VlEbDtawcLJ2vgXh65PB1XfDkcVsgsyp2pzjY6EHtA%3D%3D&amp;sigunguCd=41135&amp;bjdongCd=11000&am...

총괄표제부 데이터 조회 중...
데이터가 비어 있습니다. 재시도 중... (1/3)
데이터가 비어 있습니다. 재시도 중... (2/3)
데이터가 비어 있습니다. 재시도 중... (3/3)
총괄표제부 데이터 조회 실패 - 다른 건물 ID로 시도해 보세요

기본개요 데이터 조회 중...
데이터가 비어 있습니다. 재시도 중... (1/3)


In [1]:
# %% [markdown]
## 건축HUB API 직접 호출 예제

# %% code
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()
api_key = os.getenv("BUILDING_HUB_API_KEY")

# API 호출 함수
def call_building_hub_api(sigunguCd, bjdongCd, platGbCd, bun, ji, numOfRows=10, pageNo=1):
    """
    건축HUB API를 직접 호출하는 함수
    
    Parameters:
    -----------
    sigunguCd : str
        시군구코드 (예: '11680')
    bjdongCd : str
        법정동코드 (예: '10300')
    platGbCd : str
        대지구분코드 (예: '0')
    bun : str
        번 (예: '0012')
    ji : str
        지 (예: '0000')
    numOfRows : int
        페이지당 결과 수
    pageNo : int
        페이지 번호
    
    Returns:
    --------
    dict : API 응답 결과
    """
    base_url = "https://apis.data.go.kr/1613000/BldRgstHubService/getBrTitleInfo"
    
    params = {
        'sigunguCd': sigunguCd,
        'bjdongCd': bjdongCd,
        'platGbCd': platGbCd,
        'bun': bun,
        'ji': ji,
        'numOfRows': numOfRows,
        'pageNo': pageNo,
        '_type': 'json',
        'serviceKey': api_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None

# 예시 호출
print("건축HUB API 직접 호출 예제 실행 중...")
try:
    # 예시 파라미터
    sigunguCd = "11680"  # 강남구
    bjdongCd = "10300"   # 역삼동
    platGbCd = "0"       # 대지구분코드
    bun = "0012"         # 번
    ji = "0000"          # 지
    
    # API 호출
    result = call_building_hub_api(sigunguCd, bjdongCd, platGbCd, bun, ji)
    
    if result and 'response' in result:
        # 응답 구조 확인
        print("\nAPI 응답 구조:")
        print(json.dumps(result['response']['header'], indent=2, ensure_ascii=False))
        
        # 결과 코드 확인
        result_code = result['response']['header']['resultCode']
        result_msg = result['response']['header']['resultMsg']
        print(f"\n결과 코드: {result_code} ({result_msg})")
        
        # 데이터가 있는 경우 처리
        if 'body' in result['response'] and 'items' in result['response']['body']:
            items = result['response']['body']['items']
            if items:
                # DataFrame으로 변환
                if isinstance(items['item'], list):
                    df = pd.DataFrame(items['item'])
                else:
                    df = pd.DataFrame([items['item']])
                
                print(f"\n조회된 데이터 ({len(df)}개 레코드):")
                display(df.head())
                
                # 결과 저장
                save_path = os.path.join(notebook_dir, "data", "direct_api_result.csv")
                df.to_csv(save_path, index=False, encoding='utf-8-sig')
                print(f"\n직접 API 호출 데이터 저장 완료: {save_path}")
            else:
                print("\n조회된 데이터가 없습니다.")
        else:
            print("\n응답에 데이터가 없습니다.")
    else:
        print("\nAPI 호출 실패 또는 응답 형식이 예상과 다릅니다.")
        
except Exception as e:
    print(f"API 직접 호출 중 오류 발생: {e}")
    print("\n문제 해결 방법:")
    print("1. API 키가 올바르게 설정되었는지 확인하세요 (.env 파일)")
    print("2. 인터넷 연결을 확인하세요")
    print("3. API 서비스가 정상 작동 중인지 확인하세요")
    print("4. 파라미터가 올바른지 확인하세요")

# API URL 예시 출력
print("\n참고: API URL 예시")
print(f"https://apis.data.go.kr/1613000/BldRgstHubService/getBrTitleInfo?sigunguCd={sigunguCd}&bjdongCd={bjdongCd}&platGbCd={platGbCd}&bun={bun}&ji={ji}&numOfRows=10&pageNo=1&_type=json&serviceKey=[인증키]")


건축HUB API 직접 호출 예제 실행 중...

API 응답 구조:
{
  "resultCode": "00",
  "resultMsg": "NORMAL SERVICE"
}

결과 코드: 00 (NORMAL SERVICE)

조회된 데이터 (10개 레코드):


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,regstrGbCd,regstrGbCdNm,...,engrGrade,engrRat,engrEpi,gnBldGrade,gnBldCert,itgBldGrade,itgBldCert,crtnDay,rserthqkDsgnApplyYn,rserthqkAblty
0,1,서울특별시 강남구 개포동 12번지,11680,10300,0,0012,0000,10241912,2,집합,...,,0,0,,0,,0,20220813,1,
1,2,서울특별시 강남구 개포동 12번지,11680,10300,0,0012,0000,10241930,2,집합,...,,0,0,,0,,0,20220813,1,
2,3,서울특별시 강남구 개포동 12번지,11680,10300,0,0012,0000,10241920,2,집합,...,,0,0,,0,,0,20220813,1,
3,4,서울특별시 강남구 개포동 12번지,11680,10300,0,0012,0000,10241932,2,집합,...,,0,0,,0,,0,20220813,1,
4,5,서울특별시 강남구 개포동 12번지,11680,10300,0,0012,0000,10241914,2,집합,...,,0,0,,0,,0,20220813,1,


API 직접 호출 중 오류 발생: name 'notebook_dir' is not defined

문제 해결 방법:
1. API 키가 올바르게 설정되었는지 확인하세요 (.env 파일)
2. 인터넷 연결을 확인하세요
3. API 서비스가 정상 작동 중인지 확인하세요
4. 파라미터가 올바른지 확인하세요

참고: API URL 예시
https://apis.data.go.kr/1613000/BldRgstHubService/getBrTitleInfo?sigunguCd=11680&bjdongCd=10300&platGbCd=0&bun=0012&ji=0000&numOfRows=10&pageNo=1&_type=json&serviceKey=[인증키]


In [ ]:
# %% [markdown]
## 건축HUB API 직접 호출 예제

# %% code
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()
api_key = os.getenv("BUILDING_HUB_API_KEY")

# API 호출 함수
def call_building_hub_api(sigunguCd, bjdongCd, platGbCd, bun, ji, numOfRows=10, pageNo=1):
    """
    건축HUB API를 직접 호출하는 함수
    
    Parameters:
    -----------
    sigunguCd : str
        시군구코드 (예: '11680')
    bjdongCd : str
        법정동코드 (예: '10300')import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    "공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")

    platGbCd : str
        대지구분코드 (예: '0')
    bun : str
        번 (예: '0012')
    ji : str
        지 (예: '0000')
    numOfRows : int
        페이지당 결과 수
    pageNo : int
        페이지 번호
    
    Returns:
    --------
    dict : API 응답 결과
    """
    base_url = "https://apis.data.go.kr/1613000/BldRgstHubService/getBrTitleInfo"
    
    params = {
        'sigunguCd': sigunguCd,
        'bjdongCd': bjdongCd,
        'platGbCd': platGbCd,
        'bun': bun,
        'ji': ji,
        'numOfRows': numOfRows,
        'pageNo': pageNo,
        '_type': 'json',
        'serviceKey': api_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None

# 예시 호출
print("건축HUB API 직접 호출 예제 실행 중...")
try:
    # 예시 파라미터
    sigunguCd = "11680"  # 강남구
    bjdongCd = "10300"   # 역삼동
    platGbCd = "0"       # 대지구분코드
    bun = "0012"         # 번
    ji = "0000"          # 지
    
    # API 호출
    result = call_building_hub_api(sigunguCd, bjdongCd, platGbCd, bun, ji)
    
    if result and 'response' in result:
        # 응답 구조 확인
        print("\nAPI 응답 구조:")
        print(json.dumps(result['response']['header'], indent=2, ensure_ascii=False))
        
        # 결과 코드 확인
        result_code = result['response']['header']['resultCode']
        result_msg = result['response']['header']['resultMsg']
        print(f"\n결과 코드: {result_code} ({result_msg})")
        
        # 데이터가 있는 경우 처리
        if 'body' in result['response'] and 'items' in result['response']['body']:
            items = result['response']['body']['items']
            if items:
                # DataFrame으로 변환
                if isinstance(items['item'], list):
                    df = pd.DataFrame(items['item'])
                else:
                    df = pd.DataFrame([items['item']])
                
                print(f"\n조회된 데이터 ({len(df)}개 레코드):")
                display(df.head())
                
                # 결과 저장
                save_path = os.path.join(notebook_dir, "data", "direct_api_result.csv")
                df.to_csv(save_path, index=False, encoding='utf-8-sig')
                print(f"\n직접 API 호출 데이터 저장 완료: {save_path}")
            else:
                print("\n조회된 데이터가 없습니다.")
        else:
            print("\n응답에 데이터가 없습니다.")
    else:
        print("\nAPI 호출 실패 또는 응답 형식이 예상과 다릅니다.")
        
except Exception as e:
    print(f"API 직접 호출 중 오류 발생: {e}")
    print("\n문제 해결 방법:")
    print("1. API 키가 올바르게 설정되었는지 확인하세요 (.env 파일)")
    print("2. 인터넷 연결을 확인하세요")
    print("3. API 서비스가 정상 작동 중인지 확인하세요")
    print("4. 파라미터가 올바른지 확인하세요")

# API URL 예시 출력
print("\n참고: API URL 예시")
print(f"https://apis.data.go.kr/1613000/BldRgstHubService/getBrTitleInfo?sigunguCd={sigunguCd}&bjdongCd={bjdongCd}&platGbCd={platGbCd}&bun={bun}&ji={ji}&numOfRows=10&pageNo=1&_type=json&serviceKey=[인증키]")


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SEQ_NO               4194 non-null   int64         
 1   RECAP_PK             3938 non-null   object        
 2   연면적                  4194 non-null   float64       
 3   사용승인연도               4194 non-null   datetime64[ns]
 4   기관명                  4194 non-null   object        
 5   건축물명                 4194 non-null   object        
 6   주소                   4194 non-null   object        
 7   지상                   4189 non-null   float64       
 8   지하                   4028 non-null   float64       
 9   TOTAREA              3180 non-null   float64       
 10  BLD_NM               1982 non-null   object        
 11  DONG_NM              1733 non-null   object        
 12  USE_DATE             2978 non-null   object        
 13  MGM_BLD_PK           3179 non-nul

In [10]:
# 엑셀 파일 불러오기
import pandas as pd
import os
from collections import Counter

# 파일 경로 설정
file_path = r"C:\Users\HyeonPoong Lee\REB_GR\REB_green-remodeling-project\02_matching_Econsumption_meter\data\text_matching_result_ver3.xlsx"

# 엑셀 파일 불러오기
try:
    df = pd.read_excel(file_path)
    print(f"파일을 성공적으로 불러왔습니다. 데이터 크기: {df.shape}")
    
    # 주소 관련 컬럼 확인
    address_columns = [col for col in df.columns if '주소' in col or 'addr' in col.lower() or '지역' in col or '시군구' in col or '시도' in col]
    
    if address_columns:
        print(f"\n주소 관련 컬럼: {address_columns}")
        
        # 각 주소 컬럼별 분석
        for col in address_columns:
            print(f"\n[{col}] 컬럼 분석:")
            
            # 결측치 확인
            null_count = df[col].isna().sum()
            print(f"- 결측치 수: {null_count} ({null_count/len(df)*100:.2f}%)")
            
            # 유니크 값 개수
            unique_count = df[col].nunique()
            print(f"- 고유 값 개수: {unique_count}")
            
            # 샘플 데이터 출력
            print(f"- 샘플 데이터:")
            print(df[col].dropna().head(3).values)
            
            # 지역 분포 분석 (첫 단어 기준)
            if df[col].dtype == 'object' and not df[col].isna().all():
                # 시도 단위 추출 (첫 번째 단어 또는 '시/도/군/구' 앞까지)
                regions = []
                for addr in df[col].dropna():
                    if isinstance(addr, str):
                        # 시/도 추출 시도
                        parts = addr.split()
                        if parts:
                            # 특별시/광역시/도 패턴 확인
                            if '특별시' in parts[0] or '광역시' in parts[0] or '도' in parts[0]:
                                regions.append(parts[0])
                            elif len(parts) > 1 and ('시' in parts[1] or '군' in parts[1] or '구' in parts[1]):
                                regions.append(parts[0])
                            else:
                                regions.append(parts[0])
                
                # 지역 분포 계산
                region_counts = Counter(regions)
                print(f"- 지역 분포 (상위 10개):")
                for region, count in region_counts.most_common(10):
                    print(f"  {region}: {count}개 ({count/len(regions)*100:.2f}%)")
    else:
        print("\n주소 관련 컬럼을 찾을 수 없습니다.")
        print("모든 컬럼 목록:")
        for col in df.columns:
            print(f"- {col}")
        
    # 데이터 미리보기
    print("\n데이터 미리보기:")
    display(df.head())
    
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {file_path}")
except Exception as e:
    print(f"파일 불러오기 중 오류 발생: {e}")


파일을 성공적으로 불러왔습니다. 데이터 크기: (4194, 27)

주소 관련 컬럼: ['주소', '주소_tokens']

[주소] 컬럼 분석:
- 결측치 수: 0 (0.00%)
- 고유 값 개수: 3154
- 샘플 데이터:
['서울특별시 종로구 25-23' '서울특별시 종로구 25-23' '서울특별시 종로구 삼청동 28-102']
- 지역 분포 (상위 10개):
  서울특별시: 730개 (17.41%)
  경기도: 622개 (14.83%)
  경상남도: 328개 (7.82%)
  부산광역시: 280개 (6.68%)
  강원특별자치도: 267개 (6.37%)
  전북특별자치도: 238개 (5.67%)
  경상북도: 216개 (5.15%)
  충청북도: 215개 (5.13%)
  전라남도: 208개 (4.96%)
  대구광역시: 206개 (4.91%)

[주소_tokens] 컬럼 분석:
- 결측치 수: 2498 (59.56%)
- 고유 값 개수: 797
- 샘플 데이터:
['[]' "['학술원']" "['국사편찬위원회']"]
- 지역 분포 (상위 10개):
  []: 688개 (40.57%)
  ['금암동']: 29개 (1.71%)
  ['덕진동1가']: 18개 (1.06%)
  ['외',: 16개 (0.94%)
  ['오송생명2로',: 16개 (0.94%)
  ['신림동',: 15개 (0.88%)
  ['부산광역시',: 14개 (0.83%)
  ['산96']: 12개 (0.71%)
  ['도계읍',: 10개 (0.59%)
  ['강내면',: 10개 (0.59%)

데이터 미리보기:


,SEQ_NO,RECAP_PK,연면적,사용승인연도,기관명,건축물명,주소,지상,지하,TOTAREA,...,BD_COUNT,EBD_OVER_BD,MATCH_TOKEN_COUNT,ebd_unified_tokens,건축물명_tokens,주소_tokens,BLD_NM_tokens,DONG_NM_tokens,MATCHED_BLD_TOKENS,MATCHED_DONG_TOKENS
0,972,11110-900,10787.40,1971-04-21,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,10787.40,...,5.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,974,11110-900,5458.12,1991-07-22,감사원,제2별관(제5호),서울특별시 종로구 25-23,2.0,3.0,5458.12,...,5.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,975,11110-900,4949.30,2004-11-10,감사원,제3별관(감사원 제2별관),서울특별시 종로구 삼청동 28-102,3.0,2.0,NaN,...,5.0,no,0.0,"['감사원', '제3별관', '제2별관']","['제3별관', '감사원', '제2별관']",[],NaN,NaN,[],[]
3,976,41480-189,10615.97,1999-08-26,감사원,감사교육원(감사교육원 청사),경기도 파주시,4.0,1.0,10615.97,...,4.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1861,NaN,4948.93,1987-11-09,교육부,대한민국학술원,서울특별시 서초구 반포동 94-4 학술원,3.0,1.0,NaN,...,NaN,NaN,0.0,"['학술원', '대한민국학술원']",['대한민국학술원'],['학술원'],NaN,NaN,[],[]


In [13]:
import pandas as pd

# 1) 컬럼 정의 (테이블 정의서 순서대로)
cols = [
    "SEQ_NO",       # 공공건축물_PK
    "JI_YUK",       # 지역
    "PUR_NM",       # 용도
    "AREA",         # 면적
    "OFFICE_GB",    # 기관유형
    "CO_CD",        # 코드번호
    "OFFICE_NM",    # 기관명
    "BLD_NM",       # 건축물명
    "JUSO",         # 주소
    "SIGUNGU_CD",   # 시군구_코드
    "BJDONG_CD",    # 법정동_코드
    "SIDO_NM",      # 시도_명
    "SIGUNGU_NM",   # 시군구_명
    "BJDONG_NM",    # 법정동_명
    "PLAT_GB_CD",   # 대지_구분: 일반 0, 산 1, 블록 2
    "BUN",          # 주번지
    "JI"            # 부번지
]

# 2) TXT 파일 읽기 (sep='|', 컬럼 수 불일치시 오류 방지)
df = pd.read_csv(
    r"../data/공공건축물데이터주소추가.txt",
    sep="|",
    header=None,
    names=cols,
    dtype=str,
    engine="python",
    encoding="cp949",
    # 만약 파일 끝에 빈 줄이 있다면 skip_blank_lines=True
    skip_blank_lines=True
)

# 3) 필요하다면 데이터 정제 (예: 공백 제거)
df = df.apply(lambda col: col.str.strip())

# 4) CSV로 저장
df.to_csv("공공건축물데이터주소추가.csv", index=False, encoding="utf-8-sig")

print(f"완료: 총 {len(df)}건을 '공공건축물데이터주소추가.csv'로 저장했습니다.")


완료: 총 4194건을 '공공건축물데이터주소추가.csv'로 저장했습니다.


In [15]:
df.columns

Index(['SEQ_NO', 'JI_YUK', 'PUR_NM', 'AREA', 'OFFICE_GB', 'CO_CD', 'OFFICE_NM',
       'BLD_NM', 'JUSO', 'SIGUNGU_CD', 'BJDONG_CD', 'SIDO_NM', 'SIGUNGU_NM',
       'BJDONG_NM', 'PLAT_GB_CD', 'BUN', 'JI'],
      dtype='object')

In [47]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm  # 진행률 표시용 (선택)

# 1) .env에서 키 로드
load_dotenv()
SERVICE_KEY = os.getenv("BUILDING_HUB_API_KEY")
if not SERVICE_KEY:
    raise ValueError("환경변수 BLD_KEY가 설정되어 있지 않습니다.")

# 2) 여러분의 CSV 로드
# CSV는 미리 "sigunguCd, bjdongCd, platGbCd, bun, ji" 컬럼이 있다고 가정
#df = pd.read_csv("your_addresses.csv", dtype=str)

# 3) 호출 함수 정의
def fetch_title_row(row):
    params = {
        "serviceKey": SERVICE_KEY,
        "sigunguCd": row["SIGUNGU_CD"],   # 예: "11680"
        "bjdongCd":  row["BJDONG_CD"],    # 예: "10300"
        "platGbCd": row.get("PLAT_GB_CD","0"),  # 있을 수도/없을 수도
        "bun":       row["BUN"].zfill(4),     # "12" → "0012"
        "ji":        row["JI"].zfill(4),      # "0"  → "0000"
        "numOfRows": "100",                  # 한 번에 뽑을 최대 건수
        "pageNo":    "1",
        "_type":     "json",
    }
    r = requests.get(
        "https://apis.data.go.kr/1613000/BldRgstHubService/getBrBasisOulnInfo",
        params=params
    )
    r.raise_for_status()
    items = (r.json()
               .get("response", {})
               .get("body", {})
               .get("items", {})
               .get("item"))
    # API가 dict 하나만 줄 수도, list를 줄 수도 있으므로 통일
    if not items:
        return []
    return items if isinstance(items, list) else [items]

# 4) 전체 순회 및 결과 수집
all_records = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Collecting"):
    recs = fetch_title_row(row)
    # (만약 페이지 수를 늘려야 할 만큼 데이터가 많다면,
    #   pageNo를 1→2→… 순환하면서 추가 호출하세요.)
    all_records.extend(recs)
    time.sleep(0.1)  # 속도 조절

# 5) DataFrame 변환·저장
if all_records:
    df_out_2 = pd.DataFrame(all_records)
    df_out_2.to_csv("기본개요_건축HUB.csv", index=False, encoding="utf-8-sig")
    print("총 %d건 저장됨" % len(df_out_2))
else:
    print("조회된 데이터가 없습니다.")


Collecting: 100%|██████████| 4194/4194 [38:05<00:00,  1.83it/s]  


총 74161건 저장됨


In [52]:
# mgmBldrgstPk 변수에 하이픈(-) 추가하기
# 앞 5자리 다음에 하이픈을 넣어 형식 변경 (예: 1234566789 → 12345-66789)
df_out_2['mgmBldrgstPk'] = df_out_2['mgmBldrgstPk'].astype(str).apply(lambda x: x[:5] + '-' + x[5:] if len(x) > 5 else x)

# 변경된 결과 확인
print("하이픈이 추가된 mgmBldrgstPk 예시:")
print(df_out_2['mgmBldrgstPk'].head())


하이픈이 추가된 mgmBldrgstPk 예시:
0        11341-24083
1        11341-24082
2         11341-2380
3        11001-11317
4    11431-100228163
Name: mgmBldrgstPk, dtype: object


In [57]:
df_out_2.columns

Index(['rnum', 'platPlc', 'sigunguCd', 'bjdongCd', 'platGbCd', 'bun', 'ji',
       'mgmBldrgstPk', 'mgmUpBldrgstPk', 'bldgId', 'regstrGbCd',
       'regstrGbCdNm', 'regstrKindCd', 'regstrKindCdNm', 'newPlatPlc', 'bldNm',
       'splotNm', 'block', 'lot', 'bylotCnt', 'naRoadCd', 'naBjdongCd',
       'naUgrndCd', 'naMainBun', 'naSubBun', 'jiyukCd', 'jiguCd', 'guyukCd',
       'jiyukCdNm', 'jiguCdNm', 'guyukCdNm', 'crtnDay'],
      dtype='object')

In [61]:
df_out_2['mgmBldrgstPk'].value_counts()

mgmBldrgstPk
10491-22287                54
10491-22199                54
10491-22233                54
10491-22265                54
10491-22209                54
                           ..
12481-100202941             1
12401-11864                 1
10000-00000000002510944     1
10000-00000000002510943     1
11111-100204895             1
Name: count, Length: 12508, dtype: int64

In [64]:
# 중복 데이터 검수 및 제거
print("중복 제거 전 데이터 개수:", len(df_out_2))

# 완전 중복 확인
duplicates = df_out_2.duplicated()
print("완전 중복 데이터 개수:", duplicates.sum())

# 완전 중복 제거
df_out_2 = df_out_2.drop_duplicates()
print("중복 제거 후 데이터 개수:", len(df_out_2))

# 결과 확인
df_out_2

중복 제거 전 데이터 개수: 74161
완전 중복 데이터 개수: 61653
중복 제거 후 데이터 개수: 12508


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,mgmUpBldrgstPk,bldgId,...,naUgrndCd,naMainBun,naSubBun,jiyukCd,jiguCd,guyukCd,jiyukCdNm,jiguCdNm,guyukCdNm,crtnDay
0,1,강원특별자치도 원주시 봉산동 836-1번지,51130,11300,0,0836,0001,11341-24083,113412380,2120042320019426,...,0,134,0,,,,,,,20221112
1,2,강원특별자치도 원주시 봉산동 836-1번지,51130,11300,0,0836,0001,11341-24082,113412380,2120042320019425,...,0,134,0,,,,,,,20221112
2,3,강원특별자치도 원주시 봉산동 836-1번지,51130,11300,0,0836,0001,11341-2380,0,2020042320001321,...,0,134,0,1020,,,일반주거지역,,,20221112
3,1,경기도 동두천시 상봉암동 162-10번지,41250,10900,0,0162,0010,11001-11317,0,2120042020009040,...,0,96,63,1330,,,자연녹지지역,,,20220909
4,1,강원특별자치도 평창군 평창읍 종부리 596-4번지,51760,25025,0,0596,0004,11431-100228163,11431100228162,2120182410000083,...,0,53,0,,,,,,,20221112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74154,2,전북특별자치도 전주시 완산구 동완산동 464-1번지,52111,12200,0,0464,0001,10000-00000000004440422,0,2120242835325067,...,0,12,0,UQA430,,,자연녹지지역,,,20241015
74155,3,전북특별자치도 전주시 완산구 동완산동 464-1번지,52111,12200,0,0464,0001,11861-100359480,0,2120192830000213,...,0,464,1,UQA121,UQH110,ZL0010,제1종일반주거지역,최고고도지구,문화재보존영향 검토대상구역,20240117
74156,1,전북특별자치도 부안군 부안읍 서외리 455-41번지,52800,25022,0,0455,0041,12011-4852,0,2120042970002035,...,0,11,0,1330,,,자연녹지지역,,,20240117
74157,2,전북특별자치도 부안군 부안읍 서외리 455-41번지,52800,25022,0,0455,0041,12011-100173328,0,2120082970000450,...,0,11,0,,,,,,,20240117


In [65]:
df_out_2['mgmBldrgstPk'].value_counts()

mgmBldrgstPk
11791-2451         1
11341-24083        1
11341-24082        1
11341-2380         1
11001-11317        1
                  ..
11201-100280293    1
11201-100270852    1
11201-49503        1
11201-100344718    1
11201-100332977    1
Name: count, Length: 12508, dtype: int64

In [66]:
df_out_2[df_out_2['mgmBldrgstPk'] == "10491-22287"]


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,mgmUpBldrgstPk,bldgId,...,naUgrndCd,naMainBun,naSubBun,jiyukCd,jiguCd,guyukCd,jiyukCdNm,jiguCdNm,guyukCdNm,crtnDay
33940,100,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,0001,10491-22287,10491521,2120041490021539,...,0,80,0,,,,,,,20230219


In [56]:
df_out['bjdongCd'].value_counts()


bjdongCd
11100    6582
10800    5964
11200    5293
11700    4699
10900    4228
         ... 
37030       1
25624       1
31032       1
25629       1
31021       1
Name: count, Length: 173, dtype: int64

In [54]:
df_out['sigunguCd'].value_counts()


sigunguCd
30200    8055
27230    5476
51110    5138
48170    4476
43112    4108
         ... 
31170       2
41670       2
43740       2
47850       1
48860       1
Name: count, Length: 245, dtype: int64

In [49]:
df_out['mgmBldrgstPk'].value_counts()


mgmBldrgstPk
10491-22248        54
10491-22250        54
10491-22287        54
10491-100225580    54
10491-22295        54
                   ..
11001-680           1
11371-10133         1
11341-9627          1
11201-100293382     1
11201-49504         1
Name: count, Length: 9703, dtype: int64

In [33]:
df_out['regstrKindCdNm'].value_counts()


regstrKindCdNm
일반건축물    69884
표제부        147
Name: count, dtype: int64

In [36]:
df_out['regstrKindCd'].value_counts()


regstrKindCd
2    69884
3      147
Name: count, dtype: int64

In [34]:
df_out['regstrGbCdNm'].value_counts()


regstrGbCdNm
일반    69884
집합      147
Name: count, dtype: int64

In [35]:
df_out['regstrGbCd'].value_counts()


regstrGbCd
1    69884
2      147
Name: count, dtype: int64

In [37]:
df_out[df_out['platPlc'].str.contains('서울특별시 종로구')]


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,regstrGbCd,regstrGbCdNm,...,engrGrade,engrRat,engrEpi,gnBldGrade,gnBldCert,itgBldGrade,itgBldCert,crtnDay,rserthqkDsgnApplyYn,rserthqkAblty
4394,1,서울특별시 종로구 숭인동 201-18번지,11110,17500,0,0201,0018,10021-9953,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,
5239,1,서울특별시 종로구 동숭동 1-67번지,11110,16800,0,0001,0067,10021-100180009,2,집합,...,,0.0,0.0,,0.0,,0,20220813,0,
5420,1,서울특별시 종로구 신문로2가 2-1번지,11110,12100,0,0002,0001,10021-9408,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,
5421,2,서울특별시 종로구 신문로2가 2-1번지,11110,12100,0,0002,0001,10021-9409,1,일반,...,,0.0,0.0,,0.0,,0,20220813,1,
6849,1,서울특별시 종로구 연건동 275-1번지,11110,16600,0,0275,0001,10021-100217345,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69961,9,서울특별시 종로구 연건동 28-21번지,11110,16600,0,0028,0021,10021-100183543,1,일반,...,,0.0,0.0,,0.0,,0,20220813,1,
69962,10,서울특별시 종로구 연건동 28-21번지,11110,16600,0,0028,0021,10021-100183548,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,
69963,11,서울특별시 종로구 연건동 28-21번지,11110,16600,0,0028,0021,10021-100183542,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,
69964,12,서울특별시 종로구 연건동 28-21번지,11110,16600,0,0028,0021,10021-100183539,1,일반,...,,0.0,0.0,,0.0,,0,20220813,0,


In [22]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70031 entries, 0 to 70030
Data columns (total 78 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rnum                 70031 non-null  int64  
 1   platPlc              70031 non-null  object 
 2   sigunguCd            70031 non-null  object 
 3   bjdongCd             70031 non-null  object 
 4   platGbCd             70031 non-null  object 
 5   bun                  70031 non-null  object 
 6   ji                   70031 non-null  object 
 7   mgmBldrgstPk         70031 non-null  object 
 8   regstrGbCd           70031 non-null  object 
 9   regstrGbCdNm         70031 non-null  object 
 10  regstrKindCd         70031 non-null  object 
 11  regstrKindCdNm       70031 non-null  object 
 12  newPlatPlc           70031 non-null  object 
 13  bldNm                70031 non-null  object 
 14  splotNm              70031 non-null  object 
 15  block                70031 non-null 

In [43]:
df_out['sigunguCd'].value_counts()


sigunguCd
30200    8055
27230    5476
51110    5138
48170    4476
43112    4108
         ... 
31170       2
41670       2
43740       2
47850       1
48860       1
Name: count, Length: 245, dtype: int64

In [32]:
df_out.columns


Index(['rnum', 'platPlc', 'sigunguCd', 'bjdongCd', 'platGbCd', 'bun', 'ji',
       'mgmBldrgstPk', 'regstrGbCd', 'regstrGbCdNm', 'regstrKindCd',
       'regstrKindCdNm', 'newPlatPlc', 'bldNm', 'splotNm', 'block', 'lot',
       'bylotCnt', 'naRoadCd', 'naBjdongCd', 'naUgrndCd', 'naMainBun',
       'naSubBun', 'dongNm', 'mainAtchGbCd', 'mainAtchGbCdNm', 'platArea',
       'archArea', 'bcRat', 'totArea', 'vlRatEstmTotArea', 'vlRat', 'strctCd',
       'strctCdNm', 'etcStrct', 'mainPurpsCd', 'mainPurpsCdNm', 'etcPurps',
       'roofCd', 'roofCdNm', 'etcRoof', 'hhldCnt', 'fmlyCnt', 'heit',
       'grndFlrCnt', 'ugrndFlrCnt', 'rideUseElvtCnt', 'emgenUseElvtCnt',
       'atchBldCnt', 'atchBldArea', 'totDongTotArea', 'indrMechUtcnt',
       'indrMechArea', 'oudrMechUtcnt', 'oudrMechArea', 'indrAutoUtcnt',
       'indrAutoArea', 'oudrAutoUtcnt', 'oudrAutoArea', 'pmsDay', 'stcnsDay',
       'useAprDay', 'pmsnoYear', 'pmsnoKikCd', 'pmsnoKikCdNm', 'pmsnoGbCd',
       'pmsnoGbCdNm', 'hoCnt', 'engrGr

In [24]:
df_out['bldNm'].value_counts()


bldNm
                 32161
경북대학교             5130
부산대학교             2984
안동대학교             1482
순천대학교             1092
                 ...  
오산시립미술관              1
건강검진및뇌혈관클리닉센터        1
근로복지공단 태백병원          1
근로복지공단태백병원           1
평창문화예술회관             1
Name: count, Length: 2933, dtype: int64

In [27]:
df_out['mgmBldrgstPk'].value_counts()


mgmBldrgstPk
10491-22248        54
10491-22250        54
10491-22287        54
10491-100225580    54
10491-22295        54
                   ..
11001-680           1
11371-10133         1
11341-9627          1
11201-100293382     1
11201-49504         1
Name: count, Length: 9703, dtype: int64

In [46]:
df_out.columns

Index(['rnum', 'platPlc', 'sigunguCd', 'bjdongCd', 'platGbCd', 'bun', 'ji',
       'mgmBldrgstPk', 'regstrGbCd', 'regstrGbCdNm', 'regstrKindCd',
       'regstrKindCdNm', 'newPlatPlc', 'bldNm', 'splotNm', 'block', 'lot',
       'bylotCnt', 'naRoadCd', 'naBjdongCd', 'naUgrndCd', 'naMainBun',
       'naSubBun', 'dongNm', 'mainAtchGbCd', 'mainAtchGbCdNm', 'platArea',
       'archArea', 'bcRat', 'totArea', 'vlRatEstmTotArea', 'vlRat', 'strctCd',
       'strctCdNm', 'etcStrct', 'mainPurpsCd', 'mainPurpsCdNm', 'etcPurps',
       'roofCd', 'roofCdNm', 'etcRoof', 'hhldCnt', 'fmlyCnt', 'heit',
       'grndFlrCnt', 'ugrndFlrCnt', 'rideUseElvtCnt', 'emgenUseElvtCnt',
       'atchBldCnt', 'atchBldArea', 'totDongTotArea', 'indrMechUtcnt',
       'indrMechArea', 'oudrMechUtcnt', 'oudrMechArea', 'indrAutoUtcnt',
       'indrAutoArea', 'oudrAutoUtcnt', 'oudrAutoArea', 'pmsDay', 'stcnsDay',
       'useAprDay', 'pmsnoYear', 'pmsnoKikCd', 'pmsnoKikCdNm', 'pmsnoGbCd',
       'pmsnoGbCdNm', 'hoCnt', 'engrGr

In [68]:
df_out_2['bldgid'].value_counts()
    

KeyError: 'bldgid'

In [31]:
df_out.describe()


,rnum,bylotCnt,platArea,archArea,bcRat,totArea,vlRatEstmTotArea,vlRat,hhldCnt,fmlyCnt,...,oudrMechArea,indrAutoUtcnt,indrAutoArea,oudrAutoUtcnt,oudrAutoArea,hoCnt,engrRat,engrEpi,gnBldCert,itgBldCert
count,70031.000000,70031.00000,7.003100e+04,70031.000000,70031.000000,7.003100e+04,7.003100e+04,70031.000000,70031.000000,70031.000000,...,70031.000000,70031.000000,70031.000000,70031.000000,70031.000000,70031.000000,70031.000000,70031.000000,70031.000000,70031.0
mean,35.501963,28.48163,7.598481e+05,1329.434234,2.486894,6.767519e+03,5.313693e+03,3.206473,0.240108,0.161271,...,0.005997,23.877240,121.434010,133.477032,428.343407,0.984850,0.815838,7.097352,0.563441,0.0
std,28.392656,83.50198,5.563763e+06,5062.158179,75.760989,1.105771e+05,7.145062e+04,28.762466,5.568736,7.528516,...,0.647907,193.741491,1843.671466,551.642826,3512.435598,16.724702,11.309259,20.370742,6.001392,0.0
min,1.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,11.000000,0.00000,0.000000e+00,139.640000,0.000000,2.238000e+02,1.705600e+02,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,28.000000,6.00000,0.000000e+00,782.240000,0.000000,2.400480e+03,2.003870e+03,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,56.000000,41.00000,0.000000e+00,1541.960000,0.000000,5.893330e+03,5.259710e+03,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,100.000000,2355.00000,5.315768e+07,180316.500000,3535.000000,1.117766e+07,3.626285e+06,1126.880000,392.000000,911.000000,...,70.000000,6190.000000,181211.010000,6716.000000,144002.810000,768.000000,247.700000,185.700000,87.440000,0.0


In [ ]:
df_out['splotNm'].value_counts()


In [25]:
# mgmBldrgstPk 변수에 하이픈(-) 추가하기
# 앞 5자리 다음에 하이픈을 넣어 형식 변경 (예: 1234566789 → 12345-66789)
df_out['mgmBldrgstPk'] = df_out['mgmBldrgstPk'].astype(str).apply(lambda x: x[:5] + '-' + x[5:] if len(x) > 5 else x)

# 변경된 결과 확인
print("하이픈이 추가된 mgmBldrgstPk 예시:")
print(df_out['mgmBldrgstPk'].head())


하이픈이 추가된 mgmBldrgstPk 예시:
0        11341-24083
1        11341-24082
2        11001-11317
3    11431-100228163
4         11431-4233
Name: mgmBldrgstPk, dtype: object


In [51]:
df_out[df_out['mgmBldrgstPk'] == "11341-24083"]


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,regstrGbCd,regstrGbCdNm,...,engrGrade,engrRat,engrEpi,gnBldGrade,gnBldCert,itgBldGrade,itgBldCert,crtnDay,rserthqkDsgnApplyYn,rserthqkAblty
0,1,강원특별자치도 원주시 봉산동 836-1번지,51130,11300,0,0836,0001,11341-24083,1,일반,...,,0.0,0.0,,0.0,,0,20221112,0,
